In [1]:
import numpy as np
import tensorflow as tf
from transformers import BertTokenizer, TFBertForSequenceClassification
from keras.models import Sequential
from keras.layers import LSTM, Dense, Embedding

# Datos de ejemplo (simulados)
conversations = [
    "El cliente expresó claramente sus preocupaciones y el asesor pudo ofrecer soluciones efectivas.",
    "La conversación fue confusa y el asesor no pudo ayudar al cliente de manera satisfactoria."
]
labels = [1, 0]  # Etiquetas de calidad de la conversación (1 para alta calidad, 0 para baja calidad)

# Tokenización y padding para BERT

tokenizer = BertTokenizer.from_pretrained("dccuchile/bert-base-spanish-wwm-cased")
bert_model = TFBertForSequenceClassification.from_pretrained("dccuchile/bert-base-spanish-wwm-cased", num_labels=2)

encoded_inputs = tokenizer(conversations, padding=True, truncation=True, return_tensors="tf")
input_ids = encoded_inputs["input_ids"]

# Convertir etiquetas a tensores de TensorFlow
labels_tensor = tf.convert_to_tensor(labels)

# Modelo BER
bert_model = TFBertForSequenceClassification.from_pretrained("dccuchile/bert-base-spanish-wwm-cased", num_labels=2)
bert_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Entrenamiento del modelo BERT (usando los mismos datos simulados para simplicidad)
bert_model.fit(input_ids, labels_tensor, epochs=10, batch_size=8)

# Predicciones de BERT después del entrenamiento
bert_predictions = bert_model.predict(input_ids).logits
bert_predictions_prob = tf.nn.softmax(bert_predictions, axis=1).numpy()

# Modelo LSTM
vocab_size = 10000  # Tamaño del vocabulario
embedding_dim = 100  # Dimensión de los vectores de embedding
max_len = 50  # Longitud máxima de la secuencia
lstm_model = Sequential()
lstm_model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim))
lstm_model.add(LSTM(units=100))
lstm_model.add(Dense(1, activation='sigmoid'))
lstm_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Datos de ejemplo para LSTM (simulados)
X_train = np.random.randint(vocab_size, size=(len(conversations), max_len))
y_train = np.random.randint(2, size=len(conversations))

# Entrenar modelo LSTM
lstm_model.fit(X_train, y_train, epochs=10, batch_size=8)

# Obtener predicciones del modelo LSTM
lstm_predictions = lstm_model.predict(X_train)

# Salida final
print("Predicciones de BERT (probabilidades):")
print(bert_predictions_prob)
print("\nPredicciones de LSTM:")
print(lstm_predictions)


2024-05-31 16:54:10.470803: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Users/cero/anaconda3/envs/NPL/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/364 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/242k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/134 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/480k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/648 [00:00<?, ?B/s]

tf_model.h5:   0%|          | 0.00/537M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-cased and are newly initialized: ['classifier', 'bert/pooler/dense/bias:0', 'bert/pooler/dense/kernel:0']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-cased and are newly initialized: ['classifier', 'bert/pooler/dense/bias:0', 'bert/pooler/dense/kernel:0']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/10
Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
1/1 [==============================] - 64s 64s/step - loss: 0.6931 - accuracy: 1.0000
Epoch 2/10
1/1 [==============================] - 1s 888ms/step - loss: 0.6931 - accuracy: 0.0000e+00
Epoch 3/10
1/1 [==============================] - 1s 875ms/step - loss: 0.6931 - accuracy: 1.0000
Epoch 4/10
1/1 [==============================] - 1s 902ms/step - loss: 0.6931 - accuracy: 0.5000
Epoch 5/10
1/1 [==============================] - 1s 874ms/step - loss: 0.6931 - accuracy: 0.5000
Epoch 6/10
1/1 [==============================] - 1s 880ms/step - loss: 0.3466 - accuracy: 1.0000
Epoch 7/10
1/1 [==============================] - 1s 869ms/step - loss: 6.3224 - accuracy: 0.5000
Epoch 8/10
1/1 [===================

In [8]:
import numpy as np
import tensorflow as tf
from transformers import BertTokenizer, AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from keybert import KeyBERT
import spacy

# Datos de ejemplo (simulados)
conversations = [
    "El cliente expresó claramente sus preocupaciones sobre el rendimiento del producto y el asesor pudo ofrecer soluciones efectivas. El cliente se mostró satisfecho con la atención recibida y elogió la rapidez de la respuesta.",
    "La conversación fue confusa y el asesor no pudo ayudar al cliente de manera satisfactoria. El cliente quedó insatisfecho con la falta de soluciones ofrecidas y mencionó que considerará cambiar de proveedor.",
    "El asesor mostró empatía y comprensión hacia las necesidades del cliente, ofreciendo varias opciones de solución y explicando cada una en detalle. El cliente valoró la paciencia y el conocimiento del asesor.",
    "El cliente mencionó problemas recurrentes con el servicio, y aunque el asesor intentó ayudar, no se logró resolver la situación durante la llamada. El cliente expresó frustración y solicitó hablar con un supervisor.",
    "El cliente estaba molesto por la falta de comunicación sobre el estado de su pedido. El asesor se disculpó y ofreció un descuento en la próxima compra como compensación. El cliente aceptó la oferta y agradeció la solución rápida.",
    "El asesor proporcionó información detallada sobre las características del nuevo producto, respondiendo a todas las preguntas del cliente. El cliente expresó su interés en realizar una compra y pidió más detalles sobre el proceso de envío.",
    "La llamada fue muy positiva, con el asesor y el cliente discutiendo posibles mejoras al servicio. El cliente ofreció sugerencias constructivas y el asesor se comprometió a transmitirlas al equipo correspondiente.",
    "El cliente estaba insatisfecho con la calidad del producto recibido y solicitó un reembolso. El asesor procesó la solicitud de reembolso de inmediato y explicó los pasos a seguir. El cliente agradeció la eficiencia del proceso.",
    "El asesor ayudó al cliente a resolver un problema técnico complejo con el software. La llamada duró más de lo esperado, pero el cliente quedó muy satisfecho con la solución y elogió la profesionalidad del asesor.",
    "El cliente tenía preguntas sobre los términos y condiciones del contrato. El asesor respondió a todas las preguntas con claridad y precisión, lo que llevó al cliente a sentirse más seguro y confiado en su decisión de continuar con el servicio."
]

# Tokenizador y modelo BERT en español
tokenizer = BertTokenizer.from_pretrained("dccuchile/bert-base-spanish-wwm-cased")
encoded_inputs = tokenizer(conversations, padding=True, truncation=True, return_tensors="tf")

# Intentar cargar el modelo de resumen en español
try:
    tokenizer_summarizer = AutoTokenizer.from_pretrained("mrm8488/bert2bert_shared-spanish-finetuned-summarization")
    model_summarizer = AutoModelForSeq2SeqLM.from_pretrained("mrm8488/bert2bert_shared-spanish-finetuned-summarization")
except:
    tokenizer_summarizer = AutoTokenizer.from_pretrained("mrm8488/mbart-large-cc25-summarizer")
    model_summarizer = AutoModelForSeq2SeqLM.from_pretrained("mrm8488/mbart-large-cc25-summarizer")

# Crear el pipeline de resumen
summarizer = pipeline("summarization", model=model_summarizer, tokenizer=tokenizer_summarizer)

# Crear el modelo KeyBERT en español
kw_model = KeyBERT('dccuchile/bert-base-spanish-wwm-cased')

# Cargar el modelo de spaCy para español
nlp = spacy.load("es_core_news_sm")

for i, conversation in enumerate(conversations):
    print(f"Conversación {i+1}:")
    print(conversation)
    print("\n")

    # Resumir la conversación
    input_length = len(conversation.split())
    max_len = min(50, input_length)  # Ajustar max_length dinámicamente
    summary = summarizer(conversation, max_length=max_len, min_length=max_len // 2, do_sample=False)
    print("Resumen:", summary[0]['summary_text'])
    
    # Extraer palabras clave
    keywords = kw_model.extract_keywords(conversation, keyphrase_ngram_range=(1, 2), stop_words='spanish')
    if not keywords:
        keywords = kw_model.extract_keywords(conversation, keyphrase_ngram_range=(1, 2))
    print("Palabras clave:", keywords)
    
    # Identificar entidades nombradas
    doc = nlp(conversation)
    entities = [(ent.text, ent.label_) for ent in doc.ents]
    print("Entidades nombradas:", entities)
    print("\n" + "-"*50 + "\n")


The following encoder weights were not tied to the decoder ['bert/pooler']
The following encoder weights were not tied to the decoder ['bert/pooler']
The following encoder weights were not tied to the decoder ['bert/pooler']
The following encoder weights were not tied to the decoder ['bert/pooler']
No sentence-transformers model found with name dccuchile/bert-base-spanish-wwm-cased. Creating a new one with MEAN pooling.
Some weights of BertModel were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-cased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Conversación 1:
El cliente expresó claramente sus preocupaciones sobre el rendimiento del producto y el asesor pudo ofrecer soluciones efectivas. El cliente se mostró satisfecho con la atención recibida y elogió la rapidez de la respuesta.


Resumen: El cliente expresó bien sus preocupaciones sobre el rendimiento del producto y el asesor pudo ofrecer soluciones efectivas
Palabras clave: [('cliente expresó', 0.7601), ('del producto', 0.7481), ('cliente se', 0.7421), ('el cliente', 0.7389), ('sobre el', 0.7388)]
Entidades nombradas: [('El cliente', 'MISC')]

--------------------------------------------------

Conversación 2:
La conversación fue confusa y el asesor no pudo ayudar al cliente de manera satisfactoria. El cliente quedó insatisfecho con la falta de soluciones ofrecidas y mencionó que considerará cambiar de proveedor.


Resumen: La conversación fue confusa y el asesor no pudo ayudar al cliente de manera satisfactoria. El cliente quedó insatisfecho con la falta de soluciones ofr